In [5]:
#Using logistic regression and naive bayes classifier

# Import basic libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import ADASYN 
from collections import Counter
import seaborn as sn

# plot functions
import plot_functions as pf

# scikit packages
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
from sklearn import metrics

# settings
%matplotlib inline
sn.set_style("dark")
sn.set_palette("colorblind")


ImportError: No module named 'seaborn'

In [6]:
#loading data
df = pd.read_csv('creditcard.csv')

FileNotFoundError: [Errno 2] File b'creditcard.csv' does not exist: b'creditcard.csv'

In [7]:
#viewing sample records
df.head()

NameError: name 'df' is not defined

In [46]:
#printing number of records
print(df.shape)

(284807, 31)


In [47]:
#cheking for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [48]:
# Determine number of fraud cases in dataset

Fraud = df[df['Class'] == 1]
Valid = df[df['Class'] == 0]

outlier_fraction = len(Fraud)/float(len(Valid))
print(outlier_fraction)

print('Fraud Cases: {}'.format(len(df[df['Class'] == 1])))
print('Valid Transactions: {}'.format(len(df[df['Class'] == 0])))



0.0017304750013189597
Fraud Cases: 492
Valid Transactions: 284315


In [49]:
#separating data from labels

X = df.iloc[:,:-1]
y = df['Class']

In [50]:
#scaling data to have zero mean and unit variance

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [51]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [52]:
scaled_X

array([[-1.99658302, -0.69424232, -0.04407492, ...,  0.33089162,
        -0.06378115,  0.24496426],
       [-1.99658302,  0.60849633,  0.16117592, ..., -0.02225568,
         0.04460752, -0.34247454],
       [-1.99656197, -0.69350046, -0.81157783, ..., -0.13713686,
        -0.18102083,  1.16068593],
       ...,
       [ 1.6419735 ,  0.98002374, -0.18243372, ...,  0.01103672,
        -0.0804672 , -0.0818393 ],
       [ 1.6419735 , -0.12275539,  0.32125034, ...,  0.26960398,
         0.31668678, -0.31324853],
       [ 1.64205773, -0.27233093, -0.11489898, ..., -0.00598394,
         0.04134999,  0.51435531]])

In [53]:
# Partition data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(scaled_X,y,test_size = 0.33, random_state = 42)

In [54]:
# The dataset is unbalanced. Training unbalanced dataset with learning algorithms may lead to 
# misclassification of minority class. Therefore, to compensate for the unbalancedness
# , we will use ADASYN oversampling method as implemented in 
# imbalanced-learn package to resample the dataset.
# ADASYN (ADAptive SYNthetic) is an oversampling technique 
# that adaptively generates minority data samples according 
# to their distributions using K nearest neighbor.

ada = ADASYN(random_state = 42)
print('Original dataset shape : ', Counter(y_train))
X_res,y_res = ada.fit_sample(X_train,y_train)
print('Resampled dataset shape : ',Counter(y_res))

Original dataset shape :  Counter({0: 190477, 1: 343})
Resampled dataset shape :  Counter({0: 190477, 1: 190471})


In [55]:
#training the models
X_train,y_train = X_res,y_res

#using logistic regression
LGR_classifier = LogisticRegression()
LGR_classifier.fit(X_train,y_train)

#using Bernoulli Naive Bayes (binary classification)
BNB_classifier = BernoulliNB()
BNB_classifier.fit(X_train,y_train);

In [3]:
#evaluating models

modlist = [('Logistic Regression',LGR_classifier),('Naive Bayes Classification',BNB_classifier)]

models = [j for j in modlist]
print()

print('====================== Model Evaluation/Training Results =============================')
for i,v in models:
    scores = cross_val_score(v, X_train, y_train, cv=10)
    accuracy = metrics.accuracy_score(y_train, v.predict(X_train))
    confusion_matrix = metrics.confusion_matrix(y_train, v.predict(X_train))
    classification = metrics.classification_report(y_train, v.predict(X_train))
    print('===== {} ====='.format(i))
    print()
    print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
    print() 
    print ("Model Accuracy: ", '{}%'.format(np.round(accuracy, 3) * 100)) 
    print()
    print("Confusion Matrix:" "\n", confusion_matrix)
    print()
    print("Classification Report:" "\n", classification) 
    print()

NameError: name 'LGR_classifier' is not defined

In [4]:
# Testing models
classdict = {'NOT fraud':0, 'fraud':1}
print()
print('========================== Model Test Results ========================' "\n")   

for i, v in models:
    accuracy = metrics.accuracy_score(y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(y_test, v.predict(X_test))
    classification = metrics.classification_report(y_test, v.predict(X_test))
    print(y_test.shape)
    print('=== {} ==='.format(i))
    print ("Model Accuracy: ",  '{}%'.format(np.round(accuracy, 3) * 100))
    print()
    print("Confusion Matrix:" "\n", confusion_matrix)
    print()
    pf.plot_confusion_matrix(confusion_matrix, classes = list(classdict.keys()), title='Confusion Matrix Plot', cmap=plt.cm.summer)
    print() 
    print("Classification Report:" "\n", classification) 
    print() 


========================== Model Test Results ========================



NameError: name 'models' is not defined